In [ ]:
# Read input from a file
with open(r"C:\Users\hurua\Downloads\tilesproblem_1326658913086500.txt") as file:
    input_data = file.readlines()

# Function to extract parameters from the input
def extract_input_parameters(input_data):
    Landscape = [s.replace("  ", " x ") for s in input_data[2:22]]
    result = []
    len_line = len(Landscape[0].split())

    for j in range(0, len_line, 4):
        x, y = 0, 4
        subresult = []

        for k in range(5):
            subresult = []
            for i in range(x, y):
                try:
                    values = Landscape[i].strip().split()[j:j+4]
                    subresult.append(values)
                except:
                    print(x, y)
            result.append(subresult)
            y += 4
            x += 4

    Landscape = result

    Tiles = input_data[23:25][1].strip('{|}|\n|').split(', ')
    Tiles_Dic = {Tiles[i].split('=')[0]: Tiles[i].split('=')[1] for i in range(3)}
    Tiles = Tiles_Dic

    Targets = input_data[27:31]
    Targets_Dic = {lines[0]: lines[1] for line in Targets for lines in [line.strip('\n').split(':')]}
    Targets = {i: int(element) for i, element in enumerate(Targets_Dic.values(), 1)}

    Solution_Key = input_data[36:]

    return Landscape, Tiles, Targets, Solution_Key

# Call the function to extract input parameters
Landscape, Tiles, Targets, Solution_Key = extract_input_parameters(input_data)

# Function to check the target based on current state and path
def check_target(current_state, path):
    if path.count('L_shape') > int(Tiles['EL_SHAPE']) or path.count('Full_block') > int(Tiles['FULL_BLOCK']) or path.count('Outer_block') > int(Tiles['OUTER_BOUNDARY']):
        return 'tile_count_not_matching'

    diff = {key: Targets[key] - current_state[key] for key in Targets if key in current_state}

    if all(value == 0 for value in diff.values()):
        return 'reached'

    if min(diff.values()) < 0:
        return 'greater'

    return None

# Functions for different tile shapes
def Full_block(var):
    res = {i: 0 for i in range(1, 5)}
    return res

def Outer_block(var):
    res = {i: sum(sublist[1:3].count(str(i)) for sublist in var[1:3]) for i in range(1, 5)}
    return res

def L_shape(var):
    res = {i: sum(sublist[1:].count(str(i)) for sublist in var[1:]) for i in range(1, 5)}
    return res

# List of functions for different tile shapes
functions = [L_shape, Outer_block, Full_block]

# Variables for path and roadmap
path = []
roadmap = []

# Function to apply a given function to an area
def apply_function(funct, area):
    return funct(area)

# Recursive CSP solver function
def csp_solver(current_index, current_sum, path, roadmap, functions, Landscape):
    if current_index == len(Landscape):
        if check_target(current_sum, path) == 'reached':
            return True
        else:
            return False
    else:
        unassigned_functions = [f for f in functions]
        unassigned_functions = sorted(unassigned_functions, key=lambda f: len([v for v in f(Landscape[current_index]).values() if v > 0]))

        for func in unassigned_functions:
            path.append(func.__name__)
            applied_area = apply_function(func, Landscape[current_index])

            sum_dict = {key: current_sum[key] + applied_area[key] for key in applied_area if key in current_sum}

            if check_target(sum_dict, path) == 'greater':
                path.pop()
                continue

            if check_target(sum_dict, path) == 'tile_count_not_matching':
                path.pop()
                continue

            current_sum_total = sum_dict

            result = csp_solver(current_index + 1, current_sum_total, path, roadmap, functions, Landscape)

            if result:
                roadmap.append(f' The index : {current_index}, Area : {Landscape[current_index]}, Applied Shape : {func.__name__}, Applied shape result  : {current_sum_total} \n')
                return True
            else:
                path.pop()

        return False

# Call the CSP solver function
result = csp_solver(0, {i: 0 for i in range(1, 5)}, path, roadmap, functions, Landscape)

# Display the results
if result:
    for i in range(0, len(roadmap)):
        try:
            print(roadmap[len(roadmap) - i - 1])
        except:
            print('error', i)

    print('\n')
    print('Tiles Used Counts:', 'L shape count:', path.count('L_shape'), 'Full block count:', path.count('Full_block'), 'Outer block count:', path.count('Outer_block'))
    print('\n')
    print("Path found:", path)
else:
    print("No path found.")
